In [5]:
VERBOSE = True # pour afficher les versions des packages utilisés et des commentaires
GRID = False # if True run the gridsearch
FILE_YES= "yeah.csv"
FILE_NO = "no_not_clean_from_json.csv"

#from the last GRIDSEARCH
num_topics=4
alpha=0.31
eta=0.61

################### IMPORT ######################
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re
import sklearn
import pyLDAvis
import time
from pyLDAvis import sklearn as sklearn_lda
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
import itertools
from itertools import chain 
from langdetect import detect
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

from gensim.models import Phrases
from gensim import corpora
from gensim import models
import spacy
import pyLDAvis.gensim
nltk.download('punkt')



if VERBOSE:
    print ("Versions packages utilises")
    print ("\tpandas ",pd.__version__)
    print ("\tnumpy ",np.__version__)
    print ("\tmatplotlib",matplotlib.__version__)
    print ("\tre ",re.__version__)
    print ("\tsklearn ",sklearn.__version__)
    print ("\ttqdm 4.30.0")
    print ("\tpyLDAvis ",pyLDAvis.__version__)
    print ("\tnltk ",nltk.__version__)
    print ("\tgensim ",gensim.__version__)
    print ("\tspacy ",spacy.__version__)
tqdm.pandas()



#set the parameters
stopwords_other = ['middle', 'aged', 'author', 'followup', 'improve', 'conclusion', 'day', 'also', 'men', 
                   'background', 'patient', 'group', 'cancer', 'patients', 'groups', 'study', 'treatment', 
                   'level', 'show', 'control', 'significant', 'effects', 'effect', 'trial', 'year', 'respectively',
                   'data', 'risk', 'compare', 'clinical', 'breast', 'breasts', 'include', 'receive', 'use', 'response', 
                   'two', 'cycle', 'month', 'disease', 'week', 'woman', 'one', 'rate', 'three', 'combination', 'total',
                   'follow', 'low', 'increase', 'among', 'result', 'incidence', 'lung', 'population', 'screening', 
                   'screen', 'high', 'tumor', 'case', 'stage', 'present', 'may', 'time', 'find', 'local', 'however',
                   'evaluate', 'four', 'course', 'hight', 'give', 'test', 'age', 'subject', 'report', 'identify', 
                   'need', 'outcome', 'provide', 'information', 'support', 'well', 'design', 'assess', 'participant',
                   'child', 'cell', 'human', 'drug', 'suggest', 'induce', 'demonstrate', 'change', 'observe', 'decrease',
                   'developement', 'associate', 'expression', 'survival', 'primary', 'recieve', 'median', 'treat',
                   'difference', 'administration', 'overall', 'six', 'first','efficacy', 'administer', 'five', 'occur',
                   'without', 'every', 'range', 'objective', 'grade', 'evaluable', 'versus', 'cause', 'factor', 'type',
                   'different', 'specific', 'various', 'several', 'new', 'function', 'normal', 'although', 'involve', 
                   'cloud', 'association', 'individual', 'estimate', 'baseline', 'analysis', 'significantly', 'great',
                   'relationship', 'symptom', 'research', 'impact', 'problem', 'regard', 'knowledge', 'effective', 
                   'evidence', 'condition', 'quality', 'phase', 'agent', 'determine', 'develop', 'mean', 'see', 
                   'recurrence', 'procedure', 'operation', 'number', 'cost', 'chemotherapy', 'pain', 'toxicity',
                   'therapy', 'surgery', 'reccurence', 'radiotherapy', 'complete', 'schedule', 'phase_ii', 'could',
                   'therapeutic', 'lead', 'gene', 'mortatlity', 'risk_factor', 'death', 'mammography', 'experience',
                   'measure', 'scale', 'questionnaire', 'relate', 'concern', 'reccurrence', 'complication',
                   'management', 'program', 'protocol', 'plan', 'mortality', 'similar', 'alone', 'either', 'enrol',
                   'concentration', 'tissue', 'base', 'describe', 'role', 'system', 'discuss', 'process', 'focus', 
                   'consider', 'cervical', 'score', 'positive', 'average', 'randomized', 'within', 'site', 'image',
                   'obtain', 'area', 'exposure', 'supplement', 'old', 'whether']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Versions packages utilises
	pandas  1.0.3
	numpy  1.18.4
	matplotlib 3.2.1
	re  2.2.1
	sklearn  0.22.2.post1
	tqdm 4.30.0
	pyLDAvis  2.1.2
	nltk  3.2.5
	gensim  3.6.0
	spacy  2.2.4


In [6]:
############## READ FILE ############
df_yes = pd.read_csv(FILE_YES, sep = ',')
df_no = pd.read_csv(FILE_NO, sep = ',')
df_text_yes=df_yes.loc[:,['text', 'response','pmid']]
print ("yes",df_text_yes.head())
df_text_yes['response'].replace({1: 0}, inplace=True) 
df_text_no=df_no.loc[:,['text', 'response','pmid']]
df_text_no['response'].replace({2: 1}, inplace=True) 
print ("no",df_text_no.head())

#create a list of all pmid. We keep only lengthX_yes valus for No INM
df1 = pd.DataFrame(df_text_yes['pmid'],columns=['pmid'])
df2 = pd.DataFrame(df_text_no['pmid'][0:df_text_yes.shape[0]],columns=['pmid'])
data=pd.concat([df_text_yes,df_text_no[0:df_text_yes.shape[0]]])

print ("data",data.head())
#forcer à être une string
data.text=data.text.astype(str)

df_pmids=pd.concat([df1,df2])

#reindex the dataset in order to have the same numbers to both
df_pmids.reset_index(drop=True, inplace=True)

data.reset_index(drop=True, inplace=True)
if VERBOSE:
    print ("The data", data.head())

yes                                                 text  response      pmid
0  To examine symptom responses resulting from a ...         1  31267277
1  Increasingly, cancer is recognised as a chroni...         1  31204791
2  The objective of this study was to evaluate th...         1  30840336
3  Informal caregivers provide extended support t...         1  29310613
4  The objective of this study was to determine t...         1  28743659
no                                                 text  response      pmid
0  This is a post-hoc analysis of a prospective m...         1  30544284
1  These data demonstrate that KPT-335 has biolog...         1  30143046
2  Inhibition of endogenous pAKT by RV1001 in pri...         1  29689086
3  One hundred and seventeen patients with tamoxi...         1  15857590
4  The therapeutic results of a controlled study ...         1   1000518
data                                                 text  response      pmid
0  To examine symptom responses resulti

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
############## PREPROCESSING ###############    
if VERBOSE:
    print ("Start preprocessing ...")
# Convert the text to lowercase
data['text_processed'] = data['text'].map(lambda x: x.lower())

# Print out the first rows of papers
data['text_processed'].head()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_list = data.text_processed.values.tolist()
data_words = list(sent_to_words(data_list))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
stop_words = stopwords.words('english')
stop_words.extend(stopwords_other)

Start preprocessing ...


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]    


def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['topic_dominant', 'pourcentage_contrib', 'topic_keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [11]:
###################  RUN LDA ###################
#from the last GRIDSEARCH
num_topics=6
alpha=0.6
eta=0.01
if VERBOSE:
    %time lda_model = gensim.models.LdaMulticore(corpus=corpus,\
                                           id2word=id2word,\
                                           num_topics=num_topics, \
                                           random_state=100,\
                                           chunksize=100,\
                                           passes=10,\
                                           alpha=alpha,\
                                           eta=eta) 
else:    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,\
                                           id2word=id2word,\
                                           num_topics=num_topics, \
                                           random_state=100,\
                                           chunksize=100,\
                                           passes=10,\
                                           alpha=alpha,\
                                           eta=eta) 

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

CPU times: user 9min 37s, sys: 15.9 s, total: 9min 53s
Wall time: 9min 59s

Coherence Score:  0.4472504859719198


In [12]:
####################### PRINT THE RESULTS #############
topics = lda_model[corpus]



df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data['text'])
final_data=data.copy()
final_data = df_topic_sents_keywords.reset_index()
final_data.columns = ['pmid','topic_dominant', 'pourcentage_contrib', 'topic_keywords', 'text']
final_data = final_data.assign(pmid=df_pmids['pmid'])
final_data['response']=data['response'].values
if VERBOSE:
    print (final_data.head())
print ("\nLes topics : ")
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=40):
    print(str(i)+": "+ topic)
    print()
print ("La répartition : ")    
print (final_data.groupby(['topic_dominant','response']).size()) 

       pmid  ...  response
0  31267277  ...         0
1  31204791  ...         0
2  30840336  ...         0
3  29310613  ...         0
4  28743659  ...         0

[5 rows x 6 columns]

Les topics : 
0: 0.046*"use" + 0.032*"result" + 0.028*"study" + 0.026*"prostate" + 0.021*"trial" + 0.020*"perform" + 0.019*"cisplatin" + 0.018*"value" + 0.016*"psa" + 0.015*"compare" + 0.014*"neck" + 0.014*"method" + 0.012*"evaluate" + 0.011*"technique" + 0.011*"base" + 0.010*"obtain" + 0.010*"head" + 0.009*"sensitivity" + 0.008*"find" + 0.008*"show" + 0.008*"include" + 0.008*"evaluation" + 0.007*"biopsy" + 0.007*"finding" + 0.007*"identify" + 0.006*"model" + 0.006*"report" + 0.006*"prospective" + 0.006*"small" + 0.006*"field" + 0.005*"combine" + 0.005*"select" + 0.005*"conventional" + 0.005*"parameter" + 0.005*"calculate" + 0.005*"criterion" + 0.005*"yield" + 0.005*"large" + 0.005*"analyze" + 0.004*"review"

1: 0.023*"life" + 0.022*"compare" + 0.020*"control" + 0.019*"measure" + 0.019*"symptom" + 0.017*

In [0]:
##################### GRID SEARCH #################
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [corpus]
corpus_title = ['100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Eta': [],
                 'Coherence': []
                }
# Can take a long time to run
if GRID:
    if 1 == 1:
        pbar = tqdm(total=540)
    
        # iterate through validation corpuses
        for i in range(len(corpus_sets)):
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                                  k=k, a=a, b=b)
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Eta'].append(b)
                        model_results['Coherence'].append(cv)
                    
                        pbar.update(1)
        pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
        pbar.close()

In [0]:
#########################  APPLY BEST MODEL AFTER GRIDSEARCH ########        
if GRID:
    df = pd.read_csv("lda_tuning_results.csv", sep = ',')
    # sort the file to get the highest value of coherence
    df = df.sort_values('Coherence',ascending=False)
    df.reset_index(drop=True, inplace=True)
    idx=0
    for i in range(len(df)):
        if df['Topics'][i] != 'asymmetric' and df['Topics'][i] != 'symmetric'\
        and df['Alpha'][i] != 'asymmetric' and df['Alpha'][i] != 'symmetric'\
        and df['Eta'][i] != 'asymmetric' and df['Eta'][i] != 'symmetric':
            idx=i
            break
            
    if VERBOSE:
        print ("Best values ",df[idx:idx+1])
        num_topics=int(df['Topics'][idx])
        alpha=df['Alpha'][idx]
        alpha=float('%.e' % float(alpha))
        eta=df['Eta'][idx]
        eta=float('%.e' % float(eta))
    if VERBOSE:
        print ("num_topics",num_topics," alpha ",alpha, " eta ",eta)

        %time lda_model = gensim.models.LdaMulticore(corpus=corpus,\
                                           id2word=id2word,\
                                           num_topics=num_topics, \
                                           random_state=100,\
                                           chunksize=100,\
                                           passes=10,\
                                           alpha=alpha,\
                                           eta=eta) 
    else:
        lda_model = gensim.models.LdaMulticore(corpus=corpus,\
                                           id2word=id2word,\
                                           num_topics=num_topics, \
                                           random_state=100,\
                                           chunksize=100,\
                                           passes=10,\
                                           alpha=alpha,\
                                           eta=eta) 
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

    ####################### PRINT THE RESULTS #############
    topics = lda_model[corpus]



    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data['text'])
    final_data=data.copy()
    final_data = df_topic_sents_keywords.reset_index()
    final_data.columns = ['pmid','topic_dominant', 'pourcentage_contrib', 'topic_keywords', 'text']
    final_data = final_data.assign(pmid=df_pmids['pmid'])
    final_data['response']=data['response'].values
    if VERBOSE:
        print (final_data.head())
    print ("\nLes topics : ")
    for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=40):
        print(str(i)+": "+ topic)
        print()
    print ("La répartition : ")    
    print (final_data.groupby(['topic_dominant','response']).size()) 

In [16]:
%matplotlib inline

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

#pyLDAvis.save_html(vis, 'lda_yes_no.html')

In [0]:
pyLDAvis.save_html(vis, 'lda_GRIDSEARCH.html')